In [1]:
!pip install torch torchvision torch_snippets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyli

In [ ]:
 !git clone https://github.com/rodriguez-p-fev/UNet.git

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
from torchvision.models import vgg16_bn
import os
from datetime import datetime
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim
import torch.cuda.amp as amp

from torch_snippets import *
from torchvision import transforms

In [4]:
from UNet import UNet
from tools import label_img, blend_mask, get_data_split, get_mean_std, validate_batch
from data import get_loaders, get_measure_loaders, TestDataset
from train import set_model, train_batch

In [8]:
DATA_VERSION = 'wabtec_v2.0'
GET_MEAN_STD = False
LOAD_MODEL = True
NUM_EPOCHS = 30
BATCH_SIZE = 6
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
RAILS_CODE = 17.0

In [9]:
MODEL_PATH = "/content/drive/MyDrive/UNet/models/UNet_s5-10-15-49_epoch23_vwabtec_v2.0_loss0.067.pth"

IMG_DIR = f"/content/drive/MyDrive/datasets/{DATA_VERSION}/jpgs"
MASK_DIR = f"/content/drive/MyDrive/datasets/{DATA_VERSION}/uint8"

#trainset_path = "/content/drive/MyDrive/datasets/rs19_val/rs19_splits4000/selected_v6.5.txt"
#valset_path = "/content/drive/MyDrive/datasets/rs19_val/rs19_splits4000/selected_val.txt"


val_img_path = "/content/drive/MyDrive/datasets"
val_img_1 = "normal_16.jpg"
val_img_2 = "test2_27_16.jpg"
img_1 = np.array(Image.open(os.path.join(val_img_path,val_img_1)).convert("RGB"))
img_2 = np.array(Image.open(os.path.join(val_img_path,val_img_2)).convert("RGB"))

DATA =  sorted(os.listdir(IMG_DIR))

In [10]:
IMAGE_WIDTH = 512+256
IMAGE_HEIGHT = 512

print(IMAGE_HEIGHT,IMAGE_WIDTH)

512 768


In [11]:
if(GET_MEAN_STD):
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    X_train, X_val, y_train, y_test = train_test_split(DATA, DATA, test_size=0.05, random_state=0)
    dl = get_measure_loaders(X_train, IMG_DIR, transform)
    MEAN, STD = get_mean_std(dl)
else:
    MEAN=[0.4664, 0.4730, 0.4972]
    STD=[0.1385, 0.1469, 0.1491]
print(MEAN,STD)

[0.4664, 0.473, 0.4972] [0.1385, 0.1469, 0.1491]


In [12]:
train_transform = A.Compose([
        #A.Crop(x_min=int(ORIGINAL_WIDTH*0.3),y_min=250,x_max=int(ORIGINAL_WIDTH*0.7),y_max=ORIGINAL_HEIGHT-250,always_apply=True,p=0.2),
        #A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.HorizontalFlip(p=0.5),
        A.Rotate (limit=40, interpolation=1, border_mode=4, p=0.25),
        #A.RandomBrightnessContrast(brightness_limit=[-0.2,0.2], contrast_limit=[-0.3,0.3], p=0.1),
        #A.Blur(blur_limit=2, p=0.1),
        #A.ColorJitter(brightness=[1.0,1.0], contrast=[1.0,1.0], saturation=[0.5,2.0], hue=0.2, p=0.1),
        A.Normalize(
            mean=MEAN,
            std=STD,
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
])
val_transform = A.Compose([
        #A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=MEAN,
            std=STD,
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ])
aux_transform = A.Compose([
        #A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=MEAN,
            std=STD,
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ])

In [13]:
def training():
    session=f"{datetime.now().month}-{datetime.now().day}-{datetime.now().hour}-{datetime.now().minute}"
    LEARNING_RATE = 1e-10
    LR_DECREASE = 0.05
    WEIGHT_DECAY = 0.0025
    ref_loss = 100
    counter = 0
    print(session)

    model, criterion = set_model(LOAD_MODEL, MODEL_PATH, DEVICE)
    optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE,weight_decay=WEIGHT_DECAY)

    X_train, X_val, y_train, y_test = train_test_split(DATA, DATA, test_size=0.1, random_state=0)
    trn_dl, val_dl = get_loaders(X_train, X_val, IMG_DIR, MASK_DIR, IMG_DIR, MASK_DIR,BATCH_SIZE, train_transform, val_transform, RAILS_CODE)

    log = Report(NUM_EPOCHS)
    for ex in range(NUM_EPOCHS):
        N = len(trn_dl)
        train_epoch_losses = []
        for bx, data in enumerate(trn_dl):
            loss, acc = train_batch(model, data, optimizer, criterion, DEVICE)
            log.record(ex+(bx+1)/N,trn_loss=loss, trn_acc=acc,end='\r')
            train_epoch_losses.append(loss.item())

        N = len(val_dl)
        val_epoch_losses = []
        for bx, data in enumerate(val_dl):
            loss = validate_batch(model, data, criterion, DEVICE)
            log.record(ex+(bx+1)/N,val_loss=loss, end='\r')
            val_epoch_losses.append(loss.item())

        train_loss = np.array(train_epoch_losses).mean()
        val_loss = np.array(val_epoch_losses).mean()
        MODEL = f"/content/drive/MyDrive/UNet/models_log/UNet_s{session}_epoch{ex+1}_v{DATA_VERSION}_loss{round(val_loss,3)}.pth"
        torch.save(model.to('cpu').state_dict(), MODEL)
        state_dict = torch.load(MODEL)
        model.load_state_dict(state_dict)
        model.to(DEVICE)
        log.report_avgs(ex+1)

        if(ref_loss <= train_loss):
          counter +=1
        elif(train_loss < ref_loss):
          ref_loss = train_loss
        if(counter >= 5):
          counter = 0
          LEARNING_RATE = LEARNING_RATE*LR_DECREASE
          optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE, weight_decay=0)
          print(f'Learning rate: {LEARNING_RATE}')

        val_data = TestDataset([val_img_1], val_img_path, aux_transform)
        test_img_1, name = val_data.__getitem__(0)
        _mask_1 = label_img(test_img_1, model, img_1.shape, DEVICE)
        blended = blend_mask(img_1, _mask_1)
        img_rec_1 = Image.fromarray(blended)
        img_rec_1.save(f"{val_img_path}/results/img_epoch_{ex+1}.png")

        val_data2 = TestDataset([val_img_2], val_img_path, val_transform)
        test_img_2, name = val_data2.__getitem__(0)
        _mask_2 = label_img(test_img_2, model, img_2.shape, DEVICE)
        blended = blend_mask(img_2, _mask_2)
        img_rec_2 = Image.fromarray(blended)
        img_rec_2.save(f"{val_img_path}/results/img_2_epoch_{ex+1}.png")

In [14]:
training()

5-13-3-47


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:04<00:00, 119MB/s]


model loading
EPOCH: 1.000  trn_loss: 0.097  trn_acc: 0.925  val_loss: 0.103  (498.11s - 14445.17s remaining)
EPOCH: 2.000  trn_loss: 0.097  trn_acc: 0.926  val_loss: 0.096  (649.18s - 9088.59s remaining)
EPOCH: 3.000  trn_loss: 0.097  trn_acc: 0.925  val_loss: 0.093  (798.68s - 7188.08s remaining)
EPOCH: 4.000  trn_loss: 0.096  trn_acc: 0.926  val_loss: 0.096  (948.73s - 6166.76s remaining)
EPOCH: 5.000  trn_loss: 0.096  trn_acc: 0.926  val_loss: 0.094  (1097.71s - 5488.55s remaining)
EPOCH: 6.000  trn_loss: 0.095  trn_acc: 0.926  val_loss: 0.087  (1247.23s - 4988.91s remaining)
EPOCH: 7.000  trn_loss: 0.094  trn_acc: 0.926  val_loss: 0.092  (1396.26s - 4587.70s remaining)
EPOCH: 8.000  trn_loss: 0.095  trn_acc: 0.926  val_loss: 0.094  (1544.79s - 4248.16s remaining)
EPOCH: 9.000  trn_loss: 0.093  trn_acc: 0.926  val_loss: 0.095  (1694.01s - 3952.70s remaining)
EPOCH: 10.000  trn_loss: 0.091  trn_acc: 0.926  val_loss: 0.093  (1843.07s - 3686.14s remaining)
EPOCH: 11.000  trn_loss: 0.0